## Settings

In [1]:
import os
LoadCell_CH = 3 # Load Cell Supply Channel
Magnet_CH = 2 # Soft Electromagnet Supply Channel

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\SEMA\\"
file_path = os.path.join(folder_path, "data.csv")

In [2]:
import pyvisa
#Create Resource manager instance
rm= pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR',
 'USB0::0x0B3E::0x102C::ES000390::INSTR',
 'ASRL3::INSTR')

In [3]:
# Start the communication by VISA identifying resource name
pmx32 = rm.open_resource('USB0::0x0B3E::0x102C::ES000390::INSTR')

#Checking the machine name
print(pmx32.query('*IDN?'))

KIKUSUI,PMX32-2QU,ES000390,VER01.04 BLD0035



### definition of function

In [4]:
#通信インスタンスとチャンネルと電圧と電流を指定して出力オン
def out_put_on(sps,ch,vol,cur):
    sps.write("INST " + str(ch) + ";*OPC?")
    sps.write("VOLTage " + str(vol) + ";OPC?")
    sps.write("CURRent " + str(cur) +";OPC?")
    sps.write("OUTPut ON;OPC?")
    sps.write("INST:UNS " + str(ch) + "; *OPC?")

def out_put_on_only_V(sps,ch,vol):
    sps.write("INST " + str(ch) + ";*OPC?")
    sps.write("VOLTage " + str(vol) + ";OPC?")
    sps.write("OUTPut ON;OPC?")
    sps.write("INST:UNS " + str(ch) + "; *OPC?")

#通信インスタンスとチャンネル番号を指定して出力オフ
def out_put_off(sps,ch):
    sps.write("INST " + str(ch))
    sps.write("OUTPut OFF; *OPC?")
    sps.write("INST:UNS " + str(ch) + "; *OPC?")

## DMM6500 multimeter setting

In [5]:
import pyvisa
import time
import schedule
import pandas as pd
import serial.tools.list_ports

#Create Resource manager instance
rm = pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR',
 'USB0::0x0B3E::0x102C::ES000390::INSTR',
 'ASRL3::INSTR')

In [6]:
# Start the communication by VISA identifying resource name
dmm = rm.open_resource('USB0::0x05E6::0x6500::04537333::INSTR')

#Checking the machine name
print(dmm.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL DMM6500,04537333,1.7.11b



In [7]:
dmm.write('*RST')
dmm.write(':DISPlay:VOLTage:DIGits 3')
#dmm.write("SENS:FUNC 'VOLT:DC', (@1)")
#dmm.write("SENS:FUNC 'RES', (@6)")
#dmm.write("SENS:FUNC 'RES', (@7)")

27

### With Position

In [ ]:
# Add Row in dataframe
def add_row(df, row):
    new_row = pd.DataFrame([row], columns=df.columns)
    return pd.concat([df, new_row], ignore_index=True)

In [ ]:
#Data Frame, Multimeter, DC power supply, Total Read time, Sample Period
def df_add_read_data(df,mul,supply,total_read_time,sample_period):
    #測定時間の初期化
    start_time =  time.perf_counter()
    end_time = time.perf_counter()
    while end_time - start_time < total_read_time:
        each_start_time =  time.perf_counter()        
        dmm.write("SENS:FUNC 'VOLT:DC'")
#        dmm.write('ROUT:CLOS, (@1)')
        data1 = float(mul.query("READ?"))

        data2 = supply.query('MEAS:CURRent? (@2)')  # Measure channel 2
        data2_line = data2.splitlines()
        for line in data2_line:
            try:
                current_ch2 = float(line)
                break
            except ValueError:
                continue

        data3 = supply.query('MEAS:VOLT? (@2)')  # Measure channel 2
        data3_lines = data3.splitlines()
        for line in data3_lines:
            try:
                current_ch2 = float(line)
                break
            except ValueError:
                continue

        each_end_time = time.perf_counter()
        #サンプル周期になるまで待機
        while  each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()
            
        df = add_row(df,[each_end_time - start_time,data1,data2,data3])
        end_time = time.perf_counter()
    return df

In [ ]:
##Debug
out_put_on(pmx32,LoadCell_CH,5.0,0.1)
out_put_on(pmx32,Magnet_CH,0.5,0.2)

#### Measure

In [11]:
columns = ["Time[s]","Measured_Voltage[V]","Supply_Current[A]","Supply_Voltage[V]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 5 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,pmx32,read_time,freq)
df_voltage

C:\Users\kary\AppData\Local\Temp\ipykernel_25400\1144875871.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, new_row], ignore_index=True)


,Time[s],Measured_Voltage[V],Supply_Current[A],Supply_Voltage[V]
0,0.114349,0.000095,+2.00500E-01\n,+2.23300E-01\n
1,0.142398,0.000095,+2.00500E-01\n,+2.23800E-01\n
2,0.168497,0.000096,+2.00500E-01\n,+2.23800E-01\n
3,0.195467,0.000095,+2.00400E-01\n,+2.23800E-01\n
4,0.243494,0.000095,+2.00400E-01\n,+2.23800E-01\n
...,...,...,...,...
156,4.877215,0.000096,+2.00400E-01\n,+2.08800E-01\n
157,4.905253,0.000096,+2.00500E-01\n,+2.08800E-01\n
158,4.930316,0.000095,+2.00500E-01\n,+2.08800E-01\n
159,4.976305,0.000095,+2.00500E-01\n,+2.08800E-01\n


In [12]:
out_put_off(pmx32,2)
out_put_off(pmx32,3)